# Calculations
***

## Import Libraries 

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt

pd.set_option("display.max_rows", None)

## Data Input

In [ ]:
# Load the chlorophyll variable (chlor) from the data file
data = xr.open_dataset("https://www.oceancolour.org/thredds/dodsC/CCI_ALL-v5.0-MONTHLY?lat[0:1:4319],lon[0:1:8639],crs[0:1:279],time[0:1:279],chlor_a")
chlo = data.chlor_a.sel(lat = slice(15, -5), lon = slice(-60, -30)).load()

# Log-transform the chlorophyll (chlor) values
chlo_log = chlo.copy()
chlo_log.values = np.log10(chlo.values)

## Calculate the annual and monthly chlorophyll means

In [ ]:
# Calculate the monthly mean chlorophyll in the area
mean_monthly = chlo_log.mean(["lat", "lon"])
monthly = 10 ** mean_monthly.to_dataframe(name = "Chlorophyll-a")
monthly

In [ ]:
# Calculate the annual mean chlorophyll in the area
mean_annual = mean_monthly.groupby("time.year").mean()
annual = 10 ** mean_annual.to_dataframe(name = "Chlorophyll-a")
annual

## Calculate the monthly mean chlorophyll anomaly in the area

In [ ]:
# Deseasonalizing the data
monthly_climatology = 10 ** chlo_log.groupby("time.month").mean("time")
anomaly = chlo.groupby('time.month') - monthly_climatology

# Calculate the monthly anomaly
mean_monthly_anomaly = anomaly.mean(["lat", "lon"])
monthly_anomaly = mean_monthly_anomaly.to_series()

## Calculate the annual rates of chlorophyll change in each pixel of the area

In [ ]:
# Change the time coordinates to months and drop the extra time coordinate in anomaly
anomaly = anomaly.assign_coords(time = ("time", np.arange(0, anomaly.shape[0])))
anomaly = anomaly.reset_coords(drop = True)

# Apply a linear regression where x = time and y = chlor anomaly for each pixel
regression = anomaly.polyfit("time", 1, skipna = True, full = True)

# Calculate the annual rate of change in chloro for each pixel
total_climatology = 10 ** chlo_log.mean("time")
pixel_rates = 100 * (regression.polyfit_coefficients.sel(degree = 1) * 12) / (regression.polyfit_coefficients.sel(degree = 0) + total_climatology)

## Calculate the annual chlorophyll change rate for the whole region

In [ ]:
# Apply a linear regression where x = time and y = chlor anomaly for the mensual anomaly values
regression2 = np.polyfit(np.arange(0, anomaly.shape[0]), monthly_anomaly.values, 1)

# Calculate the annual rate of change in chloro for the region
mean_chlo = 10 ** chlo_log.mean(["lat", "lon", "time"])
total_rate = 100 * (regression2[0] * 12) / (regression2[1] + mean_chlo)

# Figures
***

## Chlorophyll concentrations in the area for a specific date

In [ ]:
date = "2020-12-01" # Date in YYYY-MM-DD, change it and display another date
mini = 0 # Minimum value of chlorophyll displayed in the figure
maxi = 10 # Maximum value of chlorophyll displayed in the figure

chlo.sel(time = date).plot(vmin = mini, vmax = maxi, size=6)
plt.tight_layout()
plt.show()

## Climatological chlorophyll concentration for the 1997-2020 period

In [ ]:
total_climatology.plot(vmin = 0, vmax = 20, size=6)
plt.title("Climatology chlorophyll concentration (1997-2020)")
plt.ylabel("Latitude (degrees)")
plt.xlabel("Longitude (degrees)")
plt.tight_layout()
plt.show()

## Annual rates of chlorophyll change (in %) for each pixel of the region

In [ ]:
pixel_rates.plot(vmin = -3, vmax = 3, size = 6, cmap = 'RdBu_r')

plt.title("Annual rates of chlorophyll change")
plt.ylabel("Latitude (degrees)")
plt.xlabel("Longitude (degrees)")
plt.tight_layout()
plt.show()

## Chlorophyll time series and trend (1997 - 2020)

In [ ]:
trend = (np.arange(0, anomaly.shape[0]) * regression2[0]) + regression2[1]

plt.figure(dpi = 150)
plt.plot(monthly.index, monthly.values, label = "Monthly regional mean")
plt.plot(monthly_anomaly.index, monthly_anomaly.values, label = "Observations - Seasonal Cycle")
plt.plot(monthly.index, trend, label = "Trend")
plt.ylim([-0.05, 0.5])
plt.title("Chlorophyll time series and trend (1997 - 2020)")
plt.ylabel("Chlorophyll-a (mg m⁻³)")
plt.xlabel("Years")
plt.legend()
plt.tight_layout()
plt.show()